In [1]:
!pip install openpyxl

You should consider upgrading via the '/Users/javierlopez/.virtualenvs/campeonatofutbol/bin/python -m pip install --upgrade pip' command.


In [2]:
import openpyxl

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from datetime import datetime
from tqdm import tqdm
from scipy.stats import poisson, skellam
from scipy.optimize import minimize
import statsmodels.api as sm
import statsmodels.formula.api as smf
from poisson_func import * 

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (16,9) 

## Simulación

In [4]:
new_sim = True
N_sim = 25000


df = pd.read_csv("matches_chile.csv")
df_tabla_2019 = pd.read_csv("Tabla2019.csv", index_col = "Equipo")
df_tabla_2019.sort_values(by=["PTS","DG"], ascending = False, inplace = True)
teams = df.Local.unique()
poisson_model = fit_poisson_model(df)
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  Goles   No. Observations:                  540
Model:                            GLM   Df Residuals:                      504
Model Family:                 Poisson   Df Model:                           35
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -751.25
Date:                Mon, 25 Jan 2021   Deviance:                       540.43
Time:                        01:06:37   Pearson chi2:                     451.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                               0.0672      0.234      0.287      0.774      -0.392       0.526
Equipo[T.Audax Italiano]                0.1191      0.219      0.543      0.587      -0.311       0.549
Equipo[T.Cobresal]                      0.0015      0.227      0.007      0.995      -0.444       0.447
Equipo[T.Colo-Colo]                    -0.2416      0.242     -0.998      0.318      -0.716       0.233
Equipo[T.Coquimbo Unido]               -0.3631      0.259     -1.401      0.161      -0.871       0.145
Equipo[T.Curicó Unido]                 -0.0954      0.234     -0.407      0.684      -0.554       0.363
Equipo[T.Everton]                      -0.1218      0.232     -0.525      0.600      -0.577       0.333
Equipo[T.Huachipato]                    0.0318      0.223      0.143      0.887      -0.405       0.469
Equipo[T.Iquique]                      -0.1476      0.240     -0.615      0.539      -0.618       0.323
Equipo[T.La Serena]                    -0.2080      0.238     -0.874      0.382      -0.674       0.258
Equipo[T.O'Higgins]                    -0.0264      0.229     -0.116      0.908      -0.474       0.422
Equipo[T.Palestino]                     0.0569      0.223      0.255      0.799      -0.380       0.494
Equipo[T.Santiago Wanderers]           -0.0201      0.227     -0.088      0.930      -0.466       0.426
Equipo[T.Universidad Católica]          0.4436      0.206      2.159      0.031       0.041       0.846
Equipo[T.Universidad de Chile]          0.0955      0.221      0.432      0.666      -0.338       0.529
Equipo[T.Universidad de Concepción]    -0.0848      0.231     -0.368      0.713      -0.537       0.367
Equipo[T.Unión Española]                0.2851      0.214      1.332      0.183      -0.134       0.705
Equipo[T.Unión La Calera]               0.3794      0.209      1.814      0.070      -0.031       0.789
Rival[T.Audax Italiano]                 0.2317      0.217      1.068      0.286      -0.194       0.657
Rival[T.Cobresal]                       0.0004      0.230      0.002      0.998      -0.451       0.452
Rival[T.Colo-Colo]                      0.0782      0.226      0.346      0.729      -0.364       0.521
Rival[T.Coquimbo Unido]                 0.0996      0.230      0.432      0.666      -0.352       0.551
Rival[T.Curicó Unido]                   0.1472      0.223      0.659      0.510      -0.290       0.585
Rival[T.Everton]                       -0.1241      0.235     -0.528      0.597      -0.585       0.337
Rival[T.Huachipato]                     0.0564      0.226      0.250      0.803      -0.387       0.500
Rival[T.Iquique]                        0.1347      0.225      0.599      0.549      -0.306       0.575
Rival[T.La Serena]                     -0.0620      0.233     -0.26

In [5]:
if new_sim == False:
    df_tabla_2020 = pd.read_csv("Tabla2020.csv", index_col = "Equipo")
    sim_poisson_local = pd.read_csv("sim_poisson_local.csv", index_col = [0,1])
    sim_poisson_local.columns = range(1, N_sim + 1)
    sim_poisson_visita = pd.read_csv("sim_poisson_visita.csv", index_col = [0,1])
    sim_poisson_visita.columns = range(1, N_sim + 1)
    df_posicion = pd.read_csv("df_posicion.csv", index_col = 0)
    df_desc_stats = pd.read_csv("df_desc_stats.csv", index_col = 0)
    summary_reasons = pd.read_excel("resumen_descenso.xlsx", index_col = 0)
    df_cd3 = pd.read_csv("df_cd3.csv", index_col = 0)
    df_rel_matches = pd.read_csv("df_rel_matches.csv", index_col = 0)
else:
    np.random.seed(42)
    #plot_poisson_dist(df)
    print("Construyendo tabla 2020")
    df_tabla_2020 = current_table(df, teams)
    df_posicion = pd.DataFrame(columns = ['Equipo', 'Tabla', 'n_sim', 'Posición'])
    print("Simulando partidos restantes del 2020")
    sim_poisson_local, sim_poisson_visita = poisson_tournament(df, poisson_model, N = N_sim)
    print("Summary positions")
    df_posicion = summary_positions(sim_poisson_local, sim_poisson_visita, N_sim, teams, df_tabla_2019,
                                        df_tabla_2020)
    print("Relegation stats")
    df_desc_stats = relegation_stats(N_sim, df_posicion, poisson_model)
    summary_reasons = create_summary_reasons(df_desc_stats, N_sim)
    df_cd3, df_rel_matches = cases_distribution(df_posicion, N_sim)

Construyendo tabla 2020
Simulando partidos restantes del 2020


36it [00:00, 63.43it/s]
  0%|          | 0/25000 [00:00<?, ?it/s]

0:00:04.127691
Summary positions


  0%|          | 0/25000 [00:00<?, ?it/s]

Relegation stats


100%|██████████| 25000/25000 [2:06:10<00:00,  3.30it/s]  


161 156 5817


100%|██████████| 25000/25000 [02:07<00:00, 196.41it/s]


A = X 0.29584
A != X 0.70416
-------------------
A = X & B = Y 0.00644
A = X & B != Y 0.2894
A != X & A = Y 0.0432
A != X & A != Y 0.66096
-------------------
A = X & B != Y & B = Z 0.00096
A = X & B != Y & B != Z 0.28844
A != X & A = Y & B = X 0.00624
A != X & A = Y & B != X 0.03696
A != X & A != Y & B = X 0.23268
A != X & A != Y & B != X 0.42828


## Prob de campeonar

In [6]:
df_posicion[(df_posicion["Posición"] == 1)&(df_posicion.Tabla == "Absoluta")].Equipo.value_counts()/N_sim

Universidad Católica    0.83440
Unión La Calera         0.16528
Unión Española          0.00032
Name: Equipo, dtype: float64

## Prob partido definición Colo Colo vs Universidad de Chile

In [7]:
df_cc_uch = df_rel_matches[(df_rel_matches.team_1.isin(["Colo-Colo","Universidad de Chile"]))&
               (df_rel_matches.team_2.isin(["Colo-Colo","Universidad de Chile"]))]
prob_definicion_uch_cc = df_cc_uch.shape[0]/N_sim
print("Porcentaje de CC vs UCH:", 100*prob_definicion_uch_cc, "%")
df_cc_uch.head()

Porcentaje de CC vs UCH: 8.304 %


,n_sim,team_1,team_2
24,25,Colo-Colo,Universidad de Chile
48,49,Colo-Colo,Universidad de Chile
72,73,Colo-Colo,Universidad de Chile
91,92,Colo-Colo,Universidad de Chile
92,93,Colo-Colo,Universidad de Chile


## Casos de descenso de UCH

#### Prob UCH último Absoluta

In [8]:
df_ult_abs = df_posicion[(df_posicion.Tabla == "Absoluta")&
            (df_posicion["Posición"] == 18)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
df_ult_abs.shape[0]/N_sim

0.0

In [9]:
df_pen_abs = df_posicion[(df_posicion.Tabla == "Absoluta")&
            (df_posicion["Posición"] == 17)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
df_pen_abs.shape[0]/N_sim

0.0

In [10]:
mod_boolean = False
if mod_boolean == True:
    team = "Universidad de Chile"
    N_matches = 5
    goles_team = 0
    goles_rival = 1
    sim_poisson_local_mod, sim_poisson_visita_mod = sim_poisson_modification(team, N_matches, goles_team, 
                                                                             goles_rival, sim_poisson_local, 
                                                                             sim_poisson_visita)
    df_posicion_mod = summary_positions(sim_poisson_local_mod, sim_poisson_visita_mod, N_sim, teams, 
                                        df_tabla_2019, df_tabla_2020)
    df_desc_stats_mod = relegation_stats(N_sim, df_posicion_mod, poisson_model)
    summary_reasons_mod = summary_reasons(df_desc_stats_mod, N_sim)
    df_cd3_mod, df_rel_matches_mod = cases_distribution(df_posicion_mod, N_sim)

#### Prob UCH último Ponderada

In [11]:
df_ult_pon = df_posicion[(df_posicion.Tabla == "Ponderada")&
            (df_posicion["Posición"] == 18)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
sim_ult_pon = df_ult_pon.n_sim
df_ult_pon.shape[0]/N_sim

0.0002

In [12]:
df_pen_pon = df_posicion[(df_posicion.Tabla == "Ponderada")&
            (df_posicion["Posición"] == 17)&
            (df_posicion["Equipo"] == "Universidad de Chile")]
sim_pen_pon = df_pen_pon.n_sim
df_pen_pon.shape[0]/N_sim

0.01004

In [13]:
summary_reasons
# ult T2020, ult TP, liguilla

,1,2,3,Prob Desc
Iquique,0.33796,0.33864,0.20804,0.88464
Universidad de Concepción,0.00108,0.65976,0.18408,0.84492
Coquimbo Unido,0.50764,0.00000,0.10640,0.61404
Colo-Colo,0.14516,0.00000,0.20416,0.34932
Universidad de Chile,0.00000,0.00160,0.16348,0.16508
Audax Italiano,0.00692,0.00000,0.04988,0.05680
Curicó Unido,0.00000,0.00000,0.03736,0.03736
La Serena,0.00096,0.00000,0.02812,0.02908
Cobresal,0.00028,0.00000,0.00620,0.00648
Everton,0.00000,0.00000,0.00608,0.00608


In [14]:
for team in teams:
    print(team)
    probs_relegation(team, df_posicion, N_sim)

Everton
O'Higgins
Antofagasta
Audax Italiano
Santiago Wanderers
Huachipato
Unión Española
Curicó Unido
Colo-Colo
Unión La Calera
Universidad de Concepción
Palestino
Universidad de Chile
Cobresal
Universidad Católica
Iquique
La Serena
Coquimbo Unido


<Figure size 1152x648 with 0 Axes>